In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
# MPC import
import numpy as np
from LinearMPC.MPCVelControl import MPCVelControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

## Partie 3.1: Contrôle de vitesse MPC

Dans cette partie, nous implémentons 4 contrôleurs MPC découplés pour :
- **X velocity** : contrôle de la vitesse selon l'axe x (états: wy, beta, vx)
- **Y velocity** : contrôle de la vitesse selon l'axe y (états: wx, alpha, vy)
- **Z velocity** : contrôle de la vitesse selon l'axe z (état: vz)
- **Roll** : contrôle de l'angle de roulis (états: wz, gamma)

**Exigences** :
- Stabilisation à vitesse nulle et angle de roulis nul
- Temps de réponse ≤ 7 secondes depuis 5 m/s (ou 40° pour roll)
- Satisfaction récursive des contraintes d'état et d'entrée

### Étape 1: Test d'un sous-système individuel (X velocity)

Commençons par tester un seul contrôleur MPC avant de les combiner tous.

In [ ]:
# Test d'un seul sous-système: X velocity
from LinearMPC_template.MPCControl_xvel import MPCControl_xvel

Ts = 0.05  # Sampling time
H = 5.0    # Horizon de 5 secondes (peut être ajusté pour tuning)

# Créer le modèle de fusée et trouver le point d'équilibre
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
xs, us = rocket.trim()
A, B = rocket.linearize(xs, us)

print(f"État d'équilibre xs: {xs}")
print(f"Entrée d'équilibre us: {us}")
print(f"\nFull A matrix shape: {A.shape}")
print(f"Full B matrix shape: {B.shape}")

# Créer le contrôleur MPC pour le sous-système X
mpc_x = MPCControl_xvel(A, B, xs, us, Ts, H)
print(f"\nSub-system X:")
print(f"  States indices: {mpc_x.x_ids} (wy, beta, vx)")
print(f"  Input indices: {mpc_x.u_ids} (dP)")
print(f"  Discretized A shape: {mpc_x.A.shape}")
print(f"  Discretized B shape: {mpc_x.B.shape}")

In [ ]:
# Test du contrôleur X avec condition initiale: vx = 5 m/s
# États du sous-système X: [wy, beta, vx]
x0_x = np.array([0.0, 0.0, 5.0])  # Départ à 5 m/s en X

# Obtenir la trajectoire optimale open-loop
u0, x_traj, u_traj = mpc_x.get_u(x0_x)

print(f"Contrôle optimal u0: {u0} rad = {np.rad2deg(u0)} deg")
print(f"État final prédit: vx = {x_traj[2, -1]:.3f} m/s")
print(f"Temps pour atteindre l'équilibre: {H:.1f} secondes")

# Visualiser la trajectoire prédite
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(10, 6))
time_horizon = np.linspace(0, H, mpc_x.N + 1)

# États
axes[0].plot(time_horizon, np.rad2deg(x_traj[0, :]), label='wy (deg/s)')
axes[0].plot(time_horizon, np.rad2deg(x_traj[1, :]), label='beta (deg)')
axes[0].plot(time_horizon, x_traj[2, :], label='vx (m/s)', linewidth=2)
axes[0].axhline(0, color='k', linestyle='--', alpha=0.3)
axes[0].set_ylabel('States')
axes[0].set_title('Open-loop MPC prediction for X subsystem')
axes[0].legend()
axes[0].grid(True)

# Entrées
time_input = np.linspace(0, H, mpc_x.N)
axes[1].step(time_input, np.rad2deg(u_traj[0, :]), where='post', label='dP (deg)')
axes[1].axhline(np.rad2deg(15), color='r', linestyle='--', alpha=0.5, label='Constraints')
axes[1].axhline(np.rad2deg(-15), color='r', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Input dP (deg)')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print(f"\n✓ Le contrôleur stabilise vx de 5 m/s à {x_traj[2, -1]:.3f} m/s en {H} secondes")

### Étape 2: Contrôle complet avec les 4 sous-systèmes

Maintenant testons le système complet avec tous les 4 contrôleurs MPC en boucle fermée.

In [ ]:
# Configuration de la simulation en boucle fermée
Ts = 0.05
sim_time = 10.0  # 10 secondes de simulation
H = 5.0          # Horizon de prédiction

# Condition initiale: vitesses non nulles selon les exigences
# États complets: [wx, wy, wz, alpha, beta, gamma, vx, vy, vz, x, y, z]
x0 = np.array([
    0, 0, 0,              # vitesses angulaires nulles
    0, 0, np.deg2rad(40), # angles: gamma = 40° (exigence pour roll)
    5.0, 5.0, 5.0,        # vitesses linéaires: 5 m/s (exigence)
    0, 0, 10              # position initiale: z=10m
])

print(f"Condition initiale x0:")
print(f"  Vitesses: vx={x0[6]:.1f}, vy={x0[7]:.1f}, vz={x0[8]:.1f} m/s")
print(f"  Angle roll: gamma={np.rad2deg(x0[5]):.1f}°")

# Créer le contrôleur MPC complet
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

print(f"\n✓ Contrôleur MPC créé avec horizon H={H}s, Ts={Ts}s")
print(f"  → {mpc.mpc_x.N} pas de temps dans l'horizon")

In [ ]:
# Simulation en boucle fermée avec le modèle linéaire
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, x_target = rocket.simulate_control(
    mpc, sim_time, H, x0, method="linear"
)

print(f"\n✓ Simulation terminée!")
print(f"  Temps simulé: {t_cl[-1]:.2f}s")
print(f"  États finaux:")
print(f"    vx: {x_cl[6, -1]:.3f} m/s (objectif: 0)")
print(f"    vy: {x_cl[7, -1]:.3f} m/s (objectif: 0)")
print(f"    vz: {x_cl[8, -1]:.3f} m/s (objectif: 0)")
print(f"    gamma: {np.rad2deg(x_cl[5, -1]):.3f}° (objectif: 0°)")

# Vérifier le temps de stabilisation (critère: 5% de la valeur initiale)
tolerance = 0.05
settling_times = {}

for name, idx, init_val in [('vx', 6, 5.0), ('vy', 7, 5.0), ('vz', 8, 5.0), ('gamma', 5, np.deg2rad(40))]:
    settled_idx = np.where(np.abs(x_cl[idx, :]) < tolerance * abs(init_val))[0]
    if len(settled_idx) > 0:
        settling_time = t_cl[settled_idx[0]]
        settling_times[name] = settling_time
        status = "✓" if settling_time <= 7.0 else "✗"
        print(f"  {status} Settling time for {name}: {settling_time:.2f}s (exigence: ≤7s)")
    else:
        print(f"  ✗ {name} n'a pas atteint l'équilibre")

In [ ]:
# Visualisation 3D et graphes temporels avec prédictions MPC
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol)

### Étape 3: Tuning et ajustement

Si les performances ne sont pas satisfaisantes, vous pouvez ajuster:

1. **Horizon H** : Augmenter pour améliorer les performances (mais augmente le coût de calcul)
2. **Matrices Q et R** dans chaque sous-système :
   - Augmenter Q pour une réponse plus rapide (mais plus agressive)
   - Augmenter R pour des entrées plus douces (mais réponse plus lente)
3. **Pas de temps Ts** : Réduire pour une meilleure précision

**Valeurs recommandées pour commencer:**
- H = 5.0s (peut être augmenté jusqu'à 10s)
- Q diagonal avec poids élevé sur les états à contrôler (vx, vy, vz, gamma)
- R avec valeurs modérées (1.0)

In [ ]:
Ts = 0.05
sim_time = ...
H = ...
x0 = ...  # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, method="linear"
)

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol);